# Multiple Linear Regression

## Importing the libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pyarrow.csv

## Importing the dataset

In [44]:
#dataset = pd.read_csv('datapoint-task-test-mini.csv', low_memory=False)
dataset = pd.read_csv('datapoint-task-full.csv', low_memory=False)

# sanity check
if list(dataset.columns[0:4]) != ['AVAILABLE NODES COUNT', 'TASK ID', 'CPU REQUIRED', 'MEMORY REQUIRED']:
    raise Exception('illegal columns')

# remove task duplicates
dataset = dataset.drop_duplicates(subset='TASK ID', keep='last')

X = dataset.iloc[:, 2:].values # from 3rd column
y = dataset.iloc[:, 0].values # first column

features_count = len(X[0])
print(f"Features count = {features_count}")

task_ids = dataset.iloc[:, 1].values # first column
print(f"Tasks count = {len(task_ids)}")

# fill in missing values with zeros
#from sklearn.impute import SimpleImputer
#imputer = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)
#imputer.fit(X)
#X = imputer.transform(X)

print(X)


Features count = 69
Tasks count = 145184
[[0.004376 0.0003975 nan ... nan nan nan]
 [0.004376 0.0003975 nan ... nan nan nan]
 [0.004376 0.0003975 nan ... nan nan nan]
 ...
 [0.0006874 0.0003109 nan ... nan nan nan]
 [0.0006874 0.0003109 nan ... nan nan nan]
 [0.0006874 0.0003109 nan ... nan nan nan]]


## Encoding categorical data

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

columns_list = [i for i in range(2, features_count)]
ct = ColumnTransformer(
    transformers=[('encoder', OneHotEncoder(), columns_list)], 
    remainder='passthrough', 
    sparse_threshold=0, 
    n_jobs=-1
)
X_cat_encoded = np.array(ct.fit_transform(X))

print(f"Encoded columns count: {len(X_cat_encoded[0])}")
print(ct.get_feature_names_out())
print(X_cat_encoded)

## Splitting the dataset into the Training set and Test set

In [ ]:
from sklearn.model_selection import train_test_split
X_test, X_train, y_test, y_train = train_test_split(X_cat_encoded, y, test_size = 0.5, random_state = 0)

In [ ]:
print(f"Trained on {len(y_train)} tasks")
print(X_train)

## Training the Multiple Linear Regression model on the Training set

In [ ]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

## Predicting the Test set results

In [ ]:
y_test_pred = regressor.predict(X_test)
np.set_printoptions(precision=2)
print(np.concatenate((y_test_pred.reshape(len(y_test_pred),1), y_test.reshape(len(y_test),1)),1))

## Predict whole set

In [ ]:
y_pred = regressor.predict(X_cat_encoded).round(10)  # round to 10 digits
print(f"Predicted {len(y_pred)} tasks")

prediction_diff = y - y_pred
print(f"Max difference = {prediction_diff.flat[np.abs(prediction_diff).argmax()]}")

In [ ]:
## Save category encoded csv

In [ ]:
dataset_cat_encoded = pd.DataFrame(X_cat_encoded, columns=ct.get_feature_names_out())

dataset_cat_encoded.insert(0, "TASK ID", task_ids)
dataset_cat_encoded.insert(1, "AVAILABLE NODES COUNT", y)
dataset_cat_encoded.insert(2, "PREDICTED AVAILABLE NODES COUNT", y_pred)

dataset_cat_encoded.to_csv('datapoint-task-category-encoded.csv', index=False)